### Geolocalisation using ChatGPT

### Packages

In [ ]:
import cv2
import base64
from openai import AzureOpenAI
import os

### Functions

In [ ]:
def geolocate_fig(file_path):
    # Define prompt
    prompt = [
        "You are a geoguesser, a person who can identify locations based on images.",
        "Given the street view image, identify the location.",
        "Please provide the following information using the json format:",
        "{'country': 'Country name', 'city': 'City name', 'latitude': 'Latitude', 'longitude': 'Longitude'}"
    ]

    prompt = " ".join(prompt)

    # Read figure
    return responde_to_fig(file_path, prompt)

def responde_to_fig(file_path, prompt):
    # Read figure
    fig = cv2.imread(file_path)

    # Encode figure
    retval, buffer = cv2.imencode(os.path.splitext(file_path_fig)[1], fig)
    if not retval:
        raise ValueError("Failed to convert image")
    fig_encoded = base64.b64encode(buffer).decode("utf-8")

    # Load environment variables
    api_version = "2024-03-01-preview"
    api_base_url = os.getenv("OPENAI_API_BASE")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

    # Initialise large language model
    model = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base_url}/deployments/{deployment_name}",
    )

    # Trigger model
    response = model.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{fig_encoded}", "detail": "high"}},
                ],
            }
        ],
        max_tokens=1000,
        temperature=0.1,
    )

    # Return response
    return response.choices[0].message.content

### Geolocalisation

In [ ]:
# %%
file_path_fig = r"c:\Users\white_rn\Documents\GitHub\image_registration\data\images\0.png"

# Describe figure
description = geolocate_fig(file_path_fig)

# Print description
print(description)